In [1]:
import os
import sys
sys_path = r"E:\\DSwithBappy_DL_Project\\MLOPS_PRO_work"

if sys_path not in sys.path:
    sys.path.append(sys_path)
    print(sys.path.append(sys_path))

None


In [14]:
import os
%pwd

'e:\\DSwithBappy_DL_Project\\MLOPS_PRO_work'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'e:\\DSwithBappy_DL_Project\\MLOPS_PRO_work'

In [15]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [16]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import create_directories

In [17]:
from pathlib import Path
from box import ConfigBox
from box.exceptions import BoxValueError
import yaml
import logging

# Ensure logger is configured
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

def read_yaml(path_to_yaml: Path) -> ConfigBox:
    try:
        with open(path_to_yaml, 'r') as yaml_file:
            content = yaml.safe_load(yaml_file)
            if content is None:  # YAML file is empty
                logger.error(f"yaml file: {path_to_yaml} is empty")
                raise ValueError("yaml file is empty")
            logger.info(f"yaml file: {path_to_yaml} loaded successfully")
            return ConfigBox(content)
    except BoxValueError as bve:
        logger.error(f"BoxValueError: {bve}")
        raise ValueError("Invalid format for ConfigBox, the content must be a mapping or iterable")
    except yaml.YAMLError as ye:
        logger.error(f"YAMLError: {ye}")
        raise ValueError("Error parsing YAML file")
    except FileNotFoundError as fnfe:
        logger.error(f"FileNotFoundError: {fnfe}")
        raise ValueError(f"The file at {path_to_yaml} was not found")
    except Exception as e:
        logger.error(f"An unexpected error occurred: {e}")
        raise e


In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [19]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [20]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [13]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-08-22 18:43:35,767: INFO: 3611462397: yaml file: config\config.yaml loaded successfully]
[2024-08-22 18:43:35,773: INFO: 3611462397: yaml file: params.yaml loaded successfully]
[2024-08-22 18:43:35,776: INFO: common: created directory at: artifacts]
[2024-08-22 18:43:35,778: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-08-22 18:43:36,570: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224,